In [1]:
!date
import pandas as pd
import subprocess
import os
import time
import csv

import sys
sys.path.append('../GOCAM_Project/dev')
import utils

Mon Sep 26 13:52:06 PDT 2022


In [2]:
pd.options.display.max_colwidth = 100

In [3]:
identifiers = pd.read_csv('../data/identifiers.csv')
identifiers = identifiers['identifier']
identifiers= identifiers[identifiers.str.contains('reacto.owl')]
identifiers.reset_index(drop=True, inplace=True)

In [4]:
def query(rIDs,uIDs,sparql_file,o_file, members2setID, setID2members):
    #rIDs = [rID,crID]
    #FORMAT QUERY VALUES STRING
    queryIDs = rIDs['crID']
    queryIDs = queryIDs.drop_duplicates()
    queryIDs = queryIDs.apply(lambda x: '<'+x+'>') #convert to IRI
    queryIDs = queryIDs.to_string(index=False)
    
        
    #RUN QUERY WITH ROBOT ON REACTO.owl
    # '&' is used for sed, not '/' as a delimiter because the identifier contains '/'
    with open(o_file, "w") as outfile:
        subprocess.run(['sed',"s&REPLACETHIS&"+queryIDs.encode("unicode_escape").decode("utf-8")+"&",sparql_file], stdout=outfile)
    subprocess.run(['robot','query','--input', '../query/reacto.owl', '--query', o_file,'../temp/results.csv'])
    
    #log_results.append(pd.read_csv('../temp/results.csv'))
    results = handle_sets('../temp/results.csv', uIDs, members2setID, setID2members)
    #log_results.append(results)
    #RESULTS ONLY HAS crID, ADD rID column
    results = map_results_to_rID(rIDs,results)
    #log_results.append(results)
    return results

In [5]:
test_list = ['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1168601',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602359',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602468',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158197',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-5359005',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-9606895',
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-3238209'
            ]
test_list

['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1168601',
 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602359',
 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602468',
 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158197',
 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-5359005',
 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-9606895',
 'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-3238209']

In [6]:
#%%time
#subprocess.run(['robot','query','--input', '../query/reacto.owl', '--query', '../temp/query_for_robot-Comp.txt','../temp/comp.csv'])
#subprocess.run(['robot','query','--input', '../query/reacto.owl', '--query', '../temp/query_for_robot-Prot.txt','../temp/results.csv'])
#subprocess.run(['robot','query','--input', '../query/reacto.owl', '--query', '../temp/query_for_robot-Set.txt','../temp/set.csv'])


In [7]:
prots = pd.read_csv('../temp/prot.csv')
sets = pd.read_csv('../temp/set.csv')
comps = pd.read_csv('../temp/comp.csv')
len(prots),len(prots.crID.unique()),len(comps),len(comps.crID.unique())

(28520, 28520, 33560, 14125)

In [8]:
sets.type.value_counts()

BioPAX type: interface org.biopax.paxtools.model.level3.Protein           21870
BioPAX type: interface org.biopax.paxtools.model.level3.PhysicalEntity     5263
BioPAX type: interface org.biopax.paxtools.model.level3.Complex            3540
BioPAX type: interface org.biopax.paxtools.model.level3.SmallMolecule      1761
BioPAX type: interface org.biopax.paxtools.model.level3.Dna                 725
BioPAX type: interface org.biopax.paxtools.model.level3.Rna                 323
Isa http://purl.obolibrary.org/obo/PR_000027744                               2
Isa http://purl.obolibrary.org/obo/PR_000000031                               2
Isa http://purl.obolibrary.org/obo/PR_000027746                               2
Name: type, dtype: int64

In [9]:
sets = sets.loc[sets.type.apply(lambda x:(x == "BioPAX type: interface org.biopax.paxtools.model.level3.Complex") or (x == "BioPAX type: interface org.biopax.paxtools.model.level3.PhysicalEntity")or (x == "BioPAX type: interface org.biopax.paxtools.model.level3.Protein"))]
len(sets),len(sets.crID.unique())

(30673, 4809)

In [10]:
set(sets.crID.values).intersection(set(prots.crID.values)),set(sets.crID.values).intersection(set(comps.crID.values)),set(comps.crID.values).intersection(set(prots.crID.values))

(set(), set(), set())

In [11]:
loc = {k:'prots' for k in prots.crID}
d2 = {k:'comps' for k in comps.crID}
d3 = {k:'sets' for k in sets.crID}
loc.update(d2)
loc.update(d3)

In [12]:
len(loc)

47454

In [13]:
comps

,crID,name,component
0,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602363,PLA2G2A:Ca2+,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602364
1,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1602363,PLA2G2A:Ca2+,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-ALL-74112
2,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-167127,HIV-1 transcription complex with (ser5) phosphorylated CTD containing extruded transcript to +30,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HIV-167131
3,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-167127,HIV-1 transcription complex with (ser5) phosphorylated CTD containing extruded transcript to +30,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-109634
4,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-167127,HIV-1 transcription complex with (ser5) phosphorylated CTD containing extruded transcript to +30,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-113404
...,...,...,...
33555,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173728,Membrane Attack Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173713
33556,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173728,Membrane Attack Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173671
33557,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173728,Membrane Attack Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173706
33558,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173728,Membrane Attack Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-173717


In [14]:
sets

,crID,name,type,entity_from_set
0,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938258
1,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938259
2,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938261
3,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938262
4,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938273,PARPs,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8938272
...,...,...,...,...
33483,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268660,Procollagen type V,BioPAX type: interface org.biopax.paxtools.model.level3.Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268645
33484,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268660,Procollagen type V,BioPAX type: interface org.biopax.paxtools.model.level3.Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268659
33485,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268660,Procollagen type V,BioPAX type: interface org.biopax.paxtools.model.level3.Complex,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-2268712
33486,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8854614,RAB33,BioPAX type: interface org.biopax.paxtools.model.level3.Protein,http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8854592


In [15]:
s1 = {'a','b'}
s2 = {'a','c'}
entities = [s1,s2]
u = set.intersection(*entities)
specifics = []
for entity in entities:
    specifics.append(entity - u)
specifics
sset = set.union(*specifics)
df = pd.DataFrame({'record':list(sset),'set_ID':'sset:'+'NAME'})
df

,record,set_ID
0,c,sset:NAME
1,b,sset:NAME


In [16]:
def recurse(rID,x,members2setID,setID2members,log):
    #log.append('searching for:'+x)
    if loc.get(x) == 'prots':
        log.append('prot:'+x)
        result = prots.query('crID == @x')
        result.record = result.record.apply(lambda x: x.split('/')[-1])
        return list(set(result.record.values))
    elif loc.get(x) == 'comps':
        log.append('comp:'+x)
        result = list(comps.query('crID == @x').component.apply(lambda y: recurse(rID, y, members2setID,setID2members, log))) 
        flat_list = list(set([item for sublist in result for item in sublist])) #once a complex is flattened, eliminate redundant entries (ie dimers)
        return flat_list
    elif loc.get(x) == 'sets':
        log.append('set:'+x)
        result = sets.query('crID == @x')
        set_name  = result.name.values[0]
        
        result = list(sets.query('crID == @x').entity_from_set.apply(lambda y: recurse(rID, y,  members2setID,setID2members, log)))
        entities = [] 
        
        for entity in result:
            entities.append(set(entity))
            
        u = set.intersection(*entities)
        specifics = []
        for entity in entities:
            specifics.append(entity - u)
            if len(entity - u) > 1:
                log.append(rID + ' ' + x)
        sset = set.union(*specifics)
        #if len(sset) > 0: FIX THIS BUG, CURRENTLY PATCHED BY REMOVING SETs with len(0) FROM SETID2members and Data_final (dict)
        setID2members['sset:'+set_name] = sset
        df = pd.DataFrame({'set_ID':'sset:'+set_name,'record':list(sset)})
        result = ['sset:'+set_name]
     
        for i in u:
            result.append(i)
        members2setID = utils.update_dict(members2setID,df,'record','set_ID') #dict of {uniprot records:{set_IDs}}
        setID2members = utils.update_dict(setID2members,df,'set_ID','record') #dict of {set_ID:{uniprot records}}
        log.append(result)
        return result
        

    else:
        if "ALL" not in x: #R-ALL instead of R-HSA signifies it is a small molecule, i think
            log.append('not found:'+x)
        return [] #this will happen for complexes with small molecules like Ca2+, for example


In [17]:
rIDs = identifiers.to_frame(name='rID')
#rIDs = rIDs.sample(n=100,random_state = 1)
#rIDs = pd.DataFrame({'rID':['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1963584']})
#rIDs = pd.DataFrame({'rID':['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-5690475']})
#rIDs = pd.DataFrame({'rID':['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-201711']})
#rIDs = pd.DataFrame({'rID':test_list})

rIDs['crID']=rIDs.rID
#rIDs=rIDs[3505:3506]
#rIDs=rIDs[rIDs['rID']=='http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158197'] #nested complex
#rIDs=rIDs[rIDs['rID']=='http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-1168601']
#rIDs = pd.DataFrame({'rID':['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158172'],'crID':['http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-158172']})
rIDs = rIDs.drop_duplicates()
uIDs = {}
members2setID = {} 
setID2members = {}
log = []
rIDs['uIDs_r']=rIDs.crID.apply(lambda x: recurse(x,x,members2setID,setID2members,log))
rIDs.uIDs_r.values[0]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


['sset:Kir heterotetramers']

In [18]:
def unpack_wrapper2(x,setID2members,setID2members_f):
    for i in x:
        if "sset:" in i:
            y = setID2members.get(i)
            new = []
            for j in y:
                if "sset:" in j:
                    new = new+ unpack(j,setID2members)
                else:
                    new.append(j)
            setID2members_f[i] = set(new)
        

In [19]:
def unpack_wrapper1(x,setID2members):
    l = list()
    for i in x:
        if "sset:" in i:
            l = l+ unpack(i,setID2members)
        else:
            l.append(i)
    return l

In [20]:
def unpack (k,d):
    vals = d.get(k)
    unpacked = []
    for v in vals:
        if "sset:" in v:
            unpacked = unpacked + unpack(v,d)
        else:
            unpacked.append(v)
    return list(set(unpacked))

In [21]:
rIDs['uIDs_f'] = rIDs.uIDs_r.apply(lambda x: unpack_wrapper1(x,setID2members))
rIDs['uIDs_f']

0       [Q92806, P48549, P63252, P78508, P48544, Q99712, P48050, Q9NPI9, Q14500, P48051]
1                                                                       [O75899, Q9UBS5]
4                                                                       [P16885, P19174]
5                                                                               [P19174]
6                                                                               [P48995]
                                              ...                                       
5295                                                    [Q8N3C0, Q8N9N2, Q96Q83, Q9H1I8]
5296                                                                            [Q96Q83]
5297                                                    [Q8N9N2, Q8N3C0, Q96Q83, Q9H1I8]
5306                                                                            [Q8TAT5]
5314                                                                            [Q9UBZ9]
Name: uIDs_f, Length:

In [22]:
setID2members_f = {}
rIDs.uIDs_r.apply(lambda x: unpack_wrapper2(x,setID2members,setID2members_f))
setID2members_f

{'sset:Kir heterotetramers': {'P48050',
  'P48051',
  'P48544',
  'P48549',
  'P63252',
  'P78508',
  'Q14500',
  'Q92806',
  'Q99712',
  'Q9NPI9'},
 'sset:p-PLC gamma1,2': {'P16885', 'P19174'},
 'sset:PLC-beta': {'Q00722', 'Q01970', 'Q15147', 'Q9NQ66'},
 'sset:G-protein alpha (q/11/14/15)': {'O95837', 'P29992', 'P30679', 'P50148'},
 'sset:IP3R subunits': {'Q14571', 'Q14573', 'Q14643'},
 'sset:ITPR tetramers': {'Q14571', 'Q14573', 'Q14643'},
 'sset:PLCdegh': {'O75038',
  'P16885',
  'P19174',
  'P51178',
  'Q4KWH8',
  'Q8N3E9',
  'Q9P212'},
 'sset:PLCbz': {'Q00722', 'Q01970', 'Q15147', 'Q86YW0', 'Q9NQ66'},
 'sset:PLC-beta 1/2/3': {'Q00722', 'Q01970', 'Q9NQ66'},
 'sset:G-protein gamma subunit': {'O14610',
  'O60262',
  'P50150',
  'P50151',
  'P59768',
  'P61952',
  'P63211',
  'P63215',
  'P63218',
  'Q9P2W3',
  'Q9UBI6',
  'Q9UK08'},
 'sset:GNB': {'O14775', 'P16520', 'P62873', 'P62879', 'Q9HAV0'},
 'sset:G(q) alpha 11,14,15,Q': {'O95837', 'P29992', 'P30679', 'P50148'},
 'sset:PLC beta

In [23]:
members2setID_f = {}
for s, m in setID2members_f.items():
    df = pd.DataFrame({'set':s,'members':list(m)})
    utils.update_dict(members2setID_f,df,'members','set')
members2setID_f    

{'Q92806': {'sset:Kir heterotetramers'},
 'P48549': {'sset:Kir heterotetramers'},
 'P63252': {'sset:Kir heterotetramers'},
 'P78508': {'sset:Kir heterotetramers'},
 'P48544': {'sset:Kir heterotetramers'},
 'Q99712': {'sset:Kir heterotetramers'},
 'P48050': {'sset:Kir heterotetramers'},
 'Q9NPI9': {'sset:Kir heterotetramers'},
 'Q14500': {'sset:Kir heterotetramers'},
 'P48051': {'sset:Kir heterotetramers'},
 'P16885': {'sset:LYN, p-SYK',
  'sset:PLCdegh',
  'sset:p-4Y-PLCG',
  'sset:p-PLC gamma1,2',
  'sset:p-Y-PLCG1,2'},
 'P19174': {'sset:PLCdegh',
  'sset:p-4Y-PLCG',
  'sset:p-PLC gamma1,2',
  'sset:p-Y-PLCG1,2'},
 'Q00722': {'sset:PLC beta1,2,3',
  'sset:PLC-beta',
  'sset:PLC-beta 1/2/3',
  'sset:PLCbz'},
 'Q15147': {'sset:PLC-beta', 'sset:PLCbz'},
 'Q9NQ66': {'sset:PLC beta1,2,3',
  'sset:PLC-beta',
  'sset:PLC-beta 1/2/3',
  'sset:PLCbz'},
 'Q01970': {'sset:PLC beta1,2,3',
  'sset:PLC-beta',
  'sset:PLC-beta 1/2/3',
  'sset:PLCbz'},
 'O95837': {'sset:G(q) alpha 11,14,15,Q', 'sset:

In [24]:
setID2members_f.get('sset:Kir heterotetramers')

{'P48050',
 'P48051',
 'P48544',
 'P48549',
 'P63252',
 'P78508',
 'Q14500',
 'Q92806',
 'Q99712',
 'Q9NPI9'}

In [25]:
rIDs[rIDs.uIDs_r.apply(lambda x: len(x) == 0)].rID.to_csv('../data/unmapped_rIDs.csv')

In [26]:
rIDs = rIDs[rIDs.uIDs_r.apply(lambda x: len(x) > 0)]

In [27]:
len(rIDs)

2992

In [28]:
'http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-453132' in identifiers.values

True

In [29]:
sets.query('crID == "http://purl.obolibrary.org/obo/go/extensions/reacto.owl#REACTO_R-HSA-8853807"')


,crID,name,type,entity_from_set


In [30]:
uIDs_f = pd.Series(rIDs.uIDs_f.values,index=rIDs.rID).to_dict()
uIDs = pd.Series(rIDs.uIDs_r.values,index=rIDs.rID).to_dict()

In [31]:
utils.dict2csv(uIDs,'../data/uIDs.csv')
utils.dict2csv(uIDs_f,'../data/uIDs_f.csv')
utils.dict2csv(members2setID_f,'../data/members2setID.csv')
utils.dict2csv(setID2members_f,'../data/setID2members.csv')

In [32]:
len(uIDs.keys()),len(identifiers.drop_duplicates()),len(identifiers)

(2992, 3076, 5315)

There are 5315 instances of reactome entitites. 3076 are unique. Of those, 84 could not be mapped. 58 of those were cases where reacto.owl does not map to any uniprot ID; often these were enzymes. 21 cases were unknown enzymes (ie 'unknown protein kinase'). 2 cases were not found in reacto.owl. 3 cases were set-like entities that were not linked through equivalent classes. 